# Geocoding and Mapping of Fire and Police Departments: Colorado Focus

- Import and clean data from multiple sources.
- Merge datasets based on the common `agency_name` column to consolidate information.
- Identify and fill in missing data from the primary dataset using supplementary data sources.
- Geocode addresses to obtain latitude and longitude coordinates.
- Export the cleaned, organized, and geocoded data to CSV files for further analysis and reporting.

In [ ]:
# Install required libraries with conda
!conda install -y pandas geopy folium -c conda-forge



In [ ]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
from folium import plugins

In [ ]:
# Store filepath in a variable
police_data = 'resources/data/police_departments.csv'
fire_data = 'resources/data/usfa-registry-national.txt'
agency_addrs = 'resources/data/27681-0001-Data.txt'
co_leads =  'resources/data/co_leads.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
police_data_df = pd.read_csv(police_data)
fire_data_df = pd.read_csv(fire_data, delimiter='\t', encoding='ISO-8859-1')
agency_addrs_df = pd.read_csv(agency_addrs, delimiter='\t', encoding='ISO-8859-1')
co_leads_df = pd.read_csv(co_leads)

In [ ]:
# Show header for DataFrames
display(police_data_df.head(), fire_data_df.head(), agency_addrs_df.head(), co_leads_df.head())


In [ ]:
# Define dictionaries for renaming columns, and standardize column names that have same data
police_rename_dict = {
    'name': 'agency_name',
    'sworn' : 'sworn_active_persnl',
    'type' : 'agency_type'
}

fire_rename_dict = {
    'Fire dept name' : 'agency_name',
    'HQ addr1' : 'hq_addr1',
    'HQ addr2' : 'hq_addr2',
    'HQ city' : 'hq_city',
    'HQ state' : 'hq_state',
    'HQ zip' : 'hq_zip',
    'Mail addr1' : 'addr1',
    'Mail addr2' : 'addr2',
    'Mail PO box' : 'po_box',
    'Mail city' : 'city',
    'Mail state' : 'state',
    'Mail zip' : 'zip',
    'HQ phone' : 'hq_ph',
    'HQ fax' : 'hq_fax',
    'County' : 'county',
    'Dept Type' : 'agency_type',
    'Organization Type' : 'org_type',
    'Website' : 'website',
    'Active Firefighters - Career' : 'sworn_active_persnl'
}

addrs_rename_dict = {
    'AGCYNAME' : 'agency_name',
    'ADDR1' : 'addr1', 
    'ADDR2' : 'addr2',
    'CITY' : 'city',
    'STATE' : 'state',
    'ZIP' : 'zip',
    'COUNTY' : 'county'
}

# Rename columns in police_data_df
police_data_df.rename(columns=police_rename_dict, inplace=True)

# Rename columns in fire_data_df
fire_data_df.rename(columns=fire_rename_dict, inplace=True)

# Rename columns in agency_addrs_df
agency_addrs_df.rename(columns=addrs_rename_dict, inplace=True)

# Verify: Display the first few rows to verify the renaming
display(police_data_df.head(), fire_data_df.head(), agency_addrs_df.head())

In [ ]:
# Drop named columns
police_data_df = police_data_df.drop(['ftciv', 'ptciv', 'ptsworn'], axis=1)
fire_data_df = fire_data_df.drop(['FDID', 'Number Of Stations', 'Active Firefighters - Volunteer', 'Active Firefighters - Paid per Call', 'Non-Firefighting - Civilian', 'Non-Firefighting - Volunteer', 'Primary agency for emergency mgmt'], axis=1)

#Save dataframe to a new file
police_data_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\police_data_clean_df.csv', index=False)
fire_data_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\fire_data_clean_df.csv', index=False)


In [ ]:
# Select several columns to keep.
columns_to_select = ['agency_name', 'addr1', 'addr2']

# Select the specified columns
agency_addrs_df = agency_addrs_df[columns_to_select]

print(agency_addrs_df.head())

In [ ]:
# For ease of commenting df1 is police_data_df and df2 is agency_addrs_df
# Test add column and iterate over rows to plug in missing data.
# Step 1: Add df2 columns to df1 with no data (initialize with empty strings)
for column in agency_addrs_df.columns:
    if column != 'agency_name':  # Avoid adding the common column again
        police_data_df[column] = ""

# For each row in df1, find matching agency_name in in df2  
# For each matching row in df2, fill in missing data in df1 columns
for index, row in police_data_df.iterrows():

    #Find df2 row where agency_name matches current row in df1
    matching_row = agency_addrs_df[agency_addrs_df['agency_name'] == row['agency_name']]
    
    if not matching_row.empty:
        # Step 3: Insert corresponding data into df1
        for column in agency_addrs_df.columns:
            if column != 'agency_name':  # Skip the common column
                police_data_df.at[index, column] = matching_row.iloc[0][column]

# Display the updated df1
print(police_data_df.head())

In [ ]:
# Sort DataFrame by agency_name in alphabetical order (A to Z)
police_data_sorted = police_data_df.sort_values(by='agency_name', ascending=True)

# Display the sorted DataFrame
display(police_data_sorted.head())

In [ ]:
# Concatenate the DataFrames vertically
merged_df = pd.concat([police_data_df, fire_data_df], ignore_index=True)

# Replace NaN values with a blank space if needed
merged_df.fillna(" ", inplace=True)

# Display the merged DataFrame
print(merged_df.head())


In [ ]:
# List the columns in the desired order
new_column_order = [
    'agency_name', 'agency_type', 'sworn_active_persnl', 'hq_addr1', 'hq_addr2',
    'hq_city', 'hq_state', 'hq_zip', 'addr1', 'addr2', 'po_box', 'city', 'state', 
    'county', 'zip', 'hq_ph', 'hq_fax', 'org_type', 'website', 'fips'
]
# Reorder the DataFrame columns
merged_df = merged_df[new_column_order]

# Display the DataFrame to check the new order
print(merged_df.head())

In [ ]:
#Save merged dataframe to a new file
merged_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\merged_data2.csv', index=False)


## Geocode Colorado Fire Deptartment Addresses


In [ ]:
# Initialize geolocator. Nominatim is good for small datasets
geolocator = Nominatim(user_agent="geoapiExercises")
